<a href="https://colab.research.google.com/github/JacekPardyak/vps/blob/master/vps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 62 kB/s 
     |████████████████████████████████| 198 kB 68.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=cd4fc1c712afdf41ee199c8b0fbddfbfe3c110345fab6c6e4382c44404aea30d
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [12]:
!wget --continue https://raw.githubusercontent.com/JacekPardyak/vps/master/data/vps_churn_data_py.json -O /tmp/vps_churn_data_py.json
!wget --continue https://raw.githubusercontent.com/JacekPardyak/vps/master/data/vps_test_data_py.json -O /tmp/vps_test_data_py.json
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('vps-randomforest').getOrCreate()
df = spark.read.json('/tmp/vps_churn_data_py.json')
df.printSchema()

--2021-09-05 20:53:03--  https://raw.githubusercontent.com/JacekPardyak/vps/master/data/vps_churn_data_py.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2021-09-05 20:53:04--  https://raw.githubusercontent.com/JacekPardyak/vps/master/data/vps_test_data_py.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

root
 |-- cpu_load_max_gradient: double (nullable = true)
 |-- cpu_load_mean_m_3:

In [5]:
df.show(5)

+---------------------+-----------------+---------------------------+-----------------------------+-------------------------+-----------------------------------+------------------------------+--------------------------+------------------------------------+--------------------------+----------------------+--------------------------------+---------------------------+-----------------------+---------------------------------+---+--------+-----------------------+-------------------+-----------------------------+-----------------------+-------------------+-----------------------------+
|cpu_load_max_gradient|cpu_load_mean_m_3|cpu_load_monthly_mean_delta|disk_octets_read_max_gradient|disk_octets_read_mean_m_3|disk_octets_read_monthly_mean_delta|disk_octets_write_max_gradient|disk_octets_write_mean_m_3|disk_octets_write_monthly_mean_delta|disk_ops_read_max_gradient|disk_ops_read_mean_m_3|disk_ops_read_monthly_mean_delta|disk_ops_write_max_gradient|disk_ops_write_mean_m_3|disk_ops_write_monthly

In [6]:
df.describe().show()

+-------+---------------------+------------------+---------------------------+-----------------------------+-------------------------+-----------------------------------+------------------------------+--------------------------+------------------------------------+--------------------------+----------------------+--------------------------------+---------------------------+-----------------------+---------------------------------+----------------+-------------------+-----------------------+-------------------+-----------------------------+-----------------------+-------------------+-----------------------------+
|summary|cpu_load_max_gradient| cpu_load_mean_m_3|cpu_load_monthly_mean_delta|disk_octets_read_max_gradient|disk_octets_read_mean_m_3|disk_octets_read_monthly_mean_delta|disk_octets_write_max_gradient|disk_octets_write_mean_m_3|disk_octets_write_monthly_mean_delta|disk_ops_read_max_gradient|disk_ops_read_mean_m_3|disk_ops_read_monthly_mean_delta|disk_ops_write_max_gradient|disk

In [11]:
df.schema.names

['cpu_load_max_gradient',
 'cpu_load_mean_m_3',
 'cpu_load_monthly_mean_delta',
 'disk_octets_read_max_gradient',
 'disk_octets_read_mean_m_3',
 'disk_octets_read_monthly_mean_delta',
 'disk_octets_write_max_gradient',
 'disk_octets_write_mean_m_3',
 'disk_octets_write_monthly_mean_delta',
 'disk_ops_read_max_gradient',
 'disk_ops_read_mean_m_3',
 'disk_ops_read_monthly_mean_delta',
 'disk_ops_write_max_gradient',
 'disk_ops_write_mean_m_3',
 'disk_ops_write_monthly_mean_delta',
 'id',
 'is_churn',
 'network_rx_max_gradient',
 'network_rx_mean_m_3',
 'network_rx_monthly_mean_delta',
 'network_tx_max_gradient',
 'network_tx_mean_m_3',
 'network_tx_monthly_mean_delta',
 'features']

In [13]:
from pyspark.ml.feature import VectorAssembler

numericCols = ['cpu_load_max_gradient',
 'cpu_load_mean_m_3',
 'cpu_load_monthly_mean_delta',
 'disk_octets_read_max_gradient',
 'disk_octets_read_mean_m_3',
 'disk_octets_read_monthly_mean_delta',
 'disk_octets_write_max_gradient',
 'disk_octets_write_mean_m_3',
 'disk_octets_write_monthly_mean_delta',
 'disk_ops_read_max_gradient',
 'disk_ops_read_mean_m_3',
 'disk_ops_read_monthly_mean_delta',
 'disk_ops_write_max_gradient',
 'disk_ops_write_mean_m_3',
 'disk_ops_write_monthly_mean_delta',
 'network_rx_max_gradient',
 'network_rx_mean_m_3',
 'network_rx_monthly_mean_delta',
 'network_tx_max_gradient',
 'network_tx_mean_m_3',
 'network_tx_monthly_mean_delta']
 
assembler = VectorAssembler(inputCols=numericCols, outputCol="features")
df = assembler.transform(df)
df.show()

+---------------------+-----------------+---------------------------+-----------------------------+-------------------------+-----------------------------------+------------------------------+--------------------------+------------------------------------+--------------------------+----------------------+--------------------------------+---------------------------+-----------------------+---------------------------------+---+--------+-----------------------+-------------------+-----------------------------+-----------------------+-------------------+-----------------------------+--------------------+
|cpu_load_max_gradient|cpu_load_mean_m_3|cpu_load_monthly_mean_delta|disk_octets_read_max_gradient|disk_octets_read_mean_m_3|disk_octets_read_monthly_mean_delta|disk_octets_write_max_gradient|disk_octets_write_mean_m_3|disk_octets_write_monthly_mean_delta|disk_ops_read_max_gradient|disk_ops_read_mean_m_3|disk_ops_read_monthly_mean_delta|disk_ops_write_max_gradient|disk_ops_write_mean_m_3|d

In [14]:
from pyspark.ml.feature import StringIndexer

label_stringIdx = StringIndexer(inputCol = 'is_churn', outputCol = 'indexedLabel')
df = label_stringIdx.fit(df).transform(df)
df.show()

+---------------------+-----------------+---------------------------+-----------------------------+-------------------------+-----------------------------------+------------------------------+--------------------------+------------------------------------+--------------------------+----------------------+--------------------------------+---------------------------+-----------------------+---------------------------------+---+--------+-----------------------+-------------------+-----------------------------+-----------------------+-------------------+-----------------------------+--------------------+------------+
|cpu_load_max_gradient|cpu_load_mean_m_3|cpu_load_monthly_mean_delta|disk_octets_read_max_gradient|disk_octets_read_mean_m_3|disk_octets_read_monthly_mean_delta|disk_octets_write_max_gradient|disk_octets_write_mean_m_3|disk_octets_write_monthly_mean_delta|disk_ops_read_max_gradient|disk_ops_read_mean_m_3|disk_ops_read_monthly_mean_delta|disk_ops_write_max_gradient|disk_ops_wri

In [15]:
train, test = df.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 195
Test Dataset Count: 88


In [16]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'indexedLabel')
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
predictions.show()

+---------------------+-----------------+---------------------------+-----------------------------+-------------------------+-----------------------------------+------------------------------+--------------------------+------------------------------------+--------------------------+----------------------+--------------------------------+---------------------------+-----------------------+---------------------------------+---+--------+-----------------------+-------------------+-----------------------------+-----------------------+-------------------+-----------------------------+--------------------+------------+--------------------+--------------------+----------+
|cpu_load_max_gradient|cpu_load_mean_m_3|cpu_load_monthly_mean_delta|disk_octets_read_max_gradient|disk_octets_read_mean_m_3|disk_octets_read_monthly_mean_delta|disk_octets_write_max_gradient|disk_octets_write_mean_m_3|disk_octets_write_monthly_mean_delta|disk_ops_read_max_gradient|disk_ops_read_mean_m_3|disk_ops_read_monthl

In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

Accuracy = 0.6046083304663495
Test Error = 0.39539166953365046


In [18]:
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F

preds_and_labels = predictions.select(['prediction','indexedLabel']).withColumn('labelIndex', F.col('indexedLabel').cast(FloatType())).orderBy('prediction')
preds_and_labels = preds_and_labels.select(['prediction','indexedLabel'])
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
print(metrics.confusionMatrix().toArray())

[[31. 20.]
 [15. 22.]]
